In [ ]:
from pyspark import SparkContext,SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
import json
import httplib
from pykafka import KafkaClient

# Create a local StreamingContext with two working thread and batch interval of 1 second
#change locat[2] to spark://server1:7077
conf = SparkConf().setAppName("Sales Order Calculation").setMaster("spark://server1:7077")
conf.set("spark.cores.max", 3)
conf.set("spark.cassandra.connection.host", "server4")
#conf.set("spark.executor.extraClassPath", "/root/spark-cassandra-connector-assembly-1.5.0-M1-SNAPSHOT.jar")
#conf.set("spark.executor.extraClassPath", "/root/spark-streaming-kafka-assembly_2.10-1.5.0.jar")
#conf.set("spark.executor.extraClassPath", "/root/kafka_2.10-0.8.2.1/libs/kafka_2.10-0.8.2.1.jar")
#conf.set("spark.executor.extraClassPath", "/root/kafka_2.10-0.8.2.1/libs/zkclient-0.3.jar")


sc = SparkContext(conf=conf)
#sc = SparkContext("spark://server1:7077", "profile_calculate")
streamingContext = StreamingContext(sc, 30)

In [ ]:
from cassandra.cluster import Cluster
from cassandra.query import BatchStatement

class SimpleClient(object):
    session = None

    def connect(self, nodes):
        cluster = Cluster(nodes)
        self.session = cluster.connect()
        self.session.execute("use test")
        
    def close(self):
        self.session.cluster.shutdown()
    
    def execute(self, statement, parameters=[]):
        preparedStatement = self.session.prepare(statement)
        return self.session.execute(preparedStatement, parameters)

In [ ]:
#change localhost:2181 to server7 or zookeeper on server1?
stream = KafkaUtils.createStream(streamingContext, 'server1.bigdata.ibm.com:2181', "profile-calculate", {'profile' :1})


def processEvent(client, producer, profile):
    if(profile.has_key('Score')):
        return
    #print '--------calculate profile----------'
    #print profile
    customer = profile['customer']
    product = profile['product']
    date = profile['date']
    
    result = client.execute("select * from customer_product where customer = ? and product = ? and date = ?", [customer, product, date])
    row = result[0]
    #print row
    
    profile['LTDSalesCount'] = 0
    profile['LTDSalesQty'] = 0
    profile['LTDSalesAmt'] = 0.0
    profile['AvgSalesAmt'] = 0.0
    profile['AvgSalesQty'] = 0.0
    
    for key in row.orders.iterkeys():
        value = row.orders.get(key)
        profile['LTDSalesCount'] += 1
        profile['LTDSalesQty'] += value.qty
        profile['LTDSalesAmt'] += value.qty * value.unitamt

    profile['AvgSalesAmt'] = profile['LTDSalesAmt'] / profile['LTDSalesQty']
    profile['AvgSalesQty'] = float(profile['LTDSalesQty']) / profile['LTDSalesCount']
    
    #TODO: calcualate score
    profile['Score'] = 1
    #qty = order["Qty"]
    #path = "/?qty=" + str(qty)
    #conn = httplib.HTTPConnection("localhost:5003")
    #conn.request("GET", path)
    #response = conn.getresponse()
    #score = response.read()
    #profile["Score"] = int(score)
    
    producer.produce([json.dumps(profile)])

    return profile
    
def processPartition(iter):
    try:
        #cassandra client
        client = SimpleClient()
        client.connect(['server4.bigdata.ibm.com','server5.bigdata.ibm.com','server6.bigdata.ibm.com'])
        #print 'connected'
        #kafka producer
        kafkaClient = KafkaClient(hosts="server7.bigdata.ibm.com:9092,server8.bigdata.ibm.com:9092,server9.bigdata.ibm.com:9092")
        topic = kafkaClient.topics['profile']
        producer = topic.get_producer()
        i = 0
        for record in iter:
            i += 1
            profile = processEvent(client, producer, json.loads(record[1]))
    finally:
        print '######## calculated ' + str(i) + ' profile ,last profile is ' +  str(profile)
        client.close()
        

stream.foreachRDD(lambda rdd: rdd.foreachPartition(processPartition))

In [ ]:
streamingContext.start()             # Start the computation
streamingContext.awaitTermination()  # Wait for the computation to terminate